In [17]:
import pandas as pd
from nbapy import game, shot_chart, player, scoreboard, team
import datetime

In [18]:
def datascrape(start):
    start = datetime.datetime.strptime(start, "%d-%m-%Y")
    #end = datetime.datetime.strptime("21-07-2014", "%d-%m-%Y")
    end = datetime.datetime.today()
    dates = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    df = pd.DataFrame({"date": dates})
    df["date"] = df["date"].astype(str)
    df[['Year', 'Month', "Day"]] = df["date"].str.split('-', 2, expand=True)
    df["Month"] = df["Month"].str.lstrip('0')
    df["Day"] = df["Day"].str.lstrip('0')
    
    #to limit to five
    #df = df.head()
    dates = []
    for i, row in df.iterrows():
        day = {}
        day["year"] = row["Year"]
        day["month"] = row["Month"]
        day["day"] = row["Day"]
        dates.append(day)
    
    init = pd.DataFrame({
    "GAME_DATE_EST_x": [],
    "GAME_ID": [],
    "TEAM_ABBREVIATION_x": [],
    "TEAM_ABBREVIATION_y": [],
    "PTS_x": [],
    "PTS_y": []
    })
    
    for date in dates:
        scores = scoreboard.Scoreboard(month=int(date["month"]), day=int(date["day"]), year=int(date["year"]), league_id='00', offset=0)
        df = scores.line_score()
        if df.empty:
            pass
        else:
            df_1 = df.drop_duplicates(subset=['GAME_ID'], keep='first')
            df_2 = df.drop_duplicates(subset=['GAME_ID'], keep='last')
            merged = df_1.merge(df_2, on=["GAME_ID"])                                                                              #FG_PCT	FT_PCT	FG3_PCT	AST	REB	TOV
            merge = merged[["GAME_DATE_EST_x", "GAME_ID", "TEAM_ABBREVIATION_x", "TEAM_ID_x", "TEAM_ID_y", "TEAM_ABBREVIATION_y", "PTS_x", "PTS_y", "FG_PCT_x", "FG_PCT_y", "FT_PCT_x", "FT_PCT_y", "FG3_PCT_x", "FG3_PCT_y", "AST_x", "AST_y", "REB_x", "REB_y", "TOV_x", "TOV_y"]]
            init = pd.concat([init, merge], sort=True)
    
    return init

In [19]:
df = datascrape("20-12-2020")


In [20]:
#Rename columns
df = df.rename(columns={
            "GAME_DATE_EST_x": "date",
            "GAME_ID": "game_id",
            "TEAM_ABBREVIATION_x": "away",
            "TEAM_ABBREVIATION_y": "home",
            "TEAM_ID_x": "away_id",
            "TEAM_ID_y": "home_id",
            "PTS_x": "away_pts",
            "PTS_y": "home_pts",
            "FG_PCT_x": "away_fg",
            "FG_PCT_y": "home_fg",
            "FT_PCT_x": "away_ft",
            "FT_PCT_y": "home_ft",
            "FG3_PCT_x": "away_fg3",
            "FG3_PCT_y": "home_fg3",
            "AST_x": "away_ast",
            "AST_y": "home_ast",
            "REB_x": "away_reb",
            "REB_y": "home_reb",
            "TOV_x": "away_tov",
            "TOV_y": "home_tov",
            })

df = df[["game_id", "date", "away", "home","away_id", "home_id","away_pts","home_pts", "away_fg","home_fg", "away_ft",
         "home_ft", "away_fg3", "home_fg3","away_ast", "home_ast", "away_reb", "home_reb", "away_tov","home_tov"]]

In [21]:
#Add catagorical for win
df.loc[df["home_pts"] > df["away_pts"], "home_win"] = 1
df.loc[df["home_pts"] < df["away_pts"], "home_win"] = 0

In [22]:
#Remove time
df["date"] = df["date"].str.replace("T00:00:00", "")

In [40]:
df = df.reset_index(drop=True).dropna()

In [52]:
df

,game_id,date,away,home,away_id,home_id,away_pts,home_pts,away_fg,home_fg,...,home_ft,away_fg3,home_fg3,away_ast,home_ast,away_reb,home_reb,away_tov,home_tov,home_win
0,0022000001,2020-12-22,GSW,BKN,1.610613e+09,1.610613e+09,99.0,125.0,0.374,0.457,...,0.813,0.303,0.429,26.0,24.0,47.0,57.0,18.0,20.0,1.0
1,0022000002,2020-12-22,LAC,LAL,1.610613e+09,1.610613e+09,116.0,109.0,0.473,0.469,...,0.774,0.350,0.310,22.0,22.0,40.0,45.0,16.0,19.0,0.0
2,0022000010,2020-12-23,CHA,CLE,1.610613e+09,1.610613e+09,114.0,121.0,0.500,0.529,...,0.750,0.364,0.467,29.0,34.0,32.0,50.0,15.0,18.0,1.0
3,0022000011,2020-12-23,NYK,IND,1.610613e+09,1.610613e+09,107.0,121.0,0.459,0.489,...,0.724,0.429,0.235,25.0,28.0,40.0,50.0,16.0,13.0,1.0
4,0022000012,2020-12-23,MIA,ORL,1.610613e+09,1.610613e+09,107.0,113.0,0.506,0.477,...,0.633,0.350,0.357,25.0,23.0,43.0,41.0,22.0,17.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0022000597,2021-03-14,CLE,ATL,1.610613e+09,1.610613e+09,82.0,100.0,0.383,0.457,...,0.923,0.267,0.414,19.0,21.0,34.0,48.0,13.0,15.0,1.0
568,0022000598,2021-03-14,BOS,HOU,1.610613e+09,1.610613e+09,134.0,107.0,0.554,0.385,...,0.677,0.486,0.320,29.0,23.0,55.0,37.0,14.0,11.0,0.0
569,0022000599,2021-03-14,POR,MIN,1.610613e+09,1.610613e+09,112.0,114.0,0.438,0.493,...,0.744,0.378,0.342,22.0,24.0,42.0,45.0,11.0,12.0,1.0
570,0022000600,2021-03-14,TOR,CHI,1.610613e+09,1.610613e+09,95.0,118.0,0.382,0.454,...,0.783,0.267,0.273,23.0,35.0,37.0,60.0,12.0,11.0,1.0


In [53]:
# Assign X (data) and y (target)
X = df[["away_fg","home_fg", "away_ft", "home_ft", "away_fg3", "home_fg3","away_ast", "home_ast", "away_reb", "home_reb", "away_tov","home_tov"]]
y = df["home_win"].values.reshape(-1, 1)
print(X.shape, y.shape)

(571, 12) (571, 1)


In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [55]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

### BEGIN SOLUTION
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [56]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
### END SOLUTION

In [57]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

LinearRegression()

LinearRegression()

In [58]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train_scaled)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_scaled)}")

Training Data Score: 0.5903905220650796
Testing Data Score: 0.5706905443449757


In [59]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [[0.32934992]
 [0.94143168]
 [0.50935306]
 [0.38255093]
 [0.27333128]
 [0.67663817]
 [0.447466  ]
 [0.31976172]
 [0.80686541]
 [0.37314363]]
First 10 Actual labels: [[0.0], [1.0], [0.0], [1.0], [0.0], [1.0], [0.0], [0.0], [1.0], [0.0]]


In [51]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

ValueError: If using all scalar values, you must pass an index